In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('../../timmy-data/loans.csv')

In [3]:
df.columns

Index(['LOAN_ID', 'LOAN_NAME', 'ORIGINAL_LANGUAGE', 'DESCRIPTION',
       'DESCRIPTION_TRANSLATED', 'FUNDED_AMOUNT', 'LOAN_AMOUNT', 'STATUS',
       'IMAGE_ID', 'VIDEO_ID', 'ACTIVITY_NAME', 'SECTOR_NAME', 'LOAN_USE',
       'COUNTRY_CODE', 'COUNTRY_NAME', 'TOWN_NAME', 'CURRENCY_POLICY',
       'CURRENCY_EXCHANGE_COVERAGE_RATE', 'CURRENCY', 'PARTNER_ID',
       'POSTED_TIME', 'PLANNED_EXPIRATION_TIME', 'DISBURSE_TIME',
       'RAISED_TIME', 'LENDER_TERM', 'NUM_LENDERS_TOTAL',
       'NUM_JOURNAL_ENTRIES', 'NUM_BULK_ENTRIES', 'TAGS', 'BORROWER_NAMES',
       'BORROWER_GENDERS', 'BORROWER_PICTURED', 'REPAYMENT_INTERVAL',
       'DISTRIBUTION_MODEL'],
      dtype='object')

In [4]:
df.columns = df.columns.str.lower()

df = df[['loan_id', 
         'status', 
         'posted_time', 
         'raised_time', 
         'loan_amount', 
         'image_id', 
         'video_id', 
         'activity_name', 
         'sector_name',
         'country_name', 
         'lender_term', 
         'repayment_interval', 
         'borrower_genders'
        ]]

In [5]:
# Dropping null raised_time; this is what we are using as our target; many
# were active campaigns; no way of imputing this; considered using disburse_time,
# but sometimes this is before the raised time, sometimes it's after
df.dropna(subset=['raised_time'], inplace=True)

# Keep only loans that have been funded
df = df[df.status == 'funded']

In [6]:
df['posted_time'] = pd.to_datetime(df['posted_time'])
df['raised_time'] = pd.to_datetime(df['raised_time'])
df['time_to_fund'] = df['raised_time'] - df['posted_time']

# https://stackoverflow.com/questions/36615565/is-it-possible-to-convert-timedelta-into-hours
df['hours_to_fund'] = np.ceil(df['time_to_fund'] / np.timedelta64(1, 'h')).astype('int64')

# https://stackoverflow.com/questions/30405413/python-pandas-extract-year-from-datetime-dfyear-dfdate-year-is-not
df['loan_year'] = df['posted_time'].dt.year

# https://stackoverflow.com/questions/30222533/create-a-day-of-week-column-in-a-pandas-dataframe-using-python
df['day_posted'] = df['posted_time'].dt.day_name()

# Drop any negative values for hours_to_fund
df = df[df.hours_to_fund >= 0]

In [7]:
# Convert image_id/video_id to binary whether included or not
df['image_incl'] = np.where(df['image_id'].isnull(), 0, 1)
df['video_incl'] = np.where(df['video_id'].isnull(), 0, 1)

In [8]:
# Assign single female borrowers to 'F', males to 'M', and any borrowers with more than one person to 'G'
df['borrower_genders'] = np.where(df['borrower_genders'] == 'female', 'F', (np.where(df['borrower_genders'] == 'male', 'M', 'G')))

In [9]:
# Dropping unneeded columns; posted_time and raised_time have been converted into
# usable data; image_id and video_id have been coded to whether present or not
df.drop(columns=['image_id', 'video_id'], inplace=True)   # 'posted_time', 'raised_time', 

# Dropping null lender_terms
df.dropna(subset=['lender_term'], inplace=True)

# Dropping any loans from countries with less than five loans historically
# https://stackoverflow.com/questions/49735683/python-removing-rows-on-count-condition
df = df.groupby('country_name').filter(lambda x : len(x)>5)

In [10]:
df.to_csv('../../timmy-data/loans_cleaned.csv', index=False)